In [8]:
!pip install dash
!pip install plotly

In [1]:
# Importing required libraries
import dash
#from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from datetime import date
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from scipy.optimize import curve_fit


C:\Users\ashdh\AppData\Local\Temp\ipykernel_14424\1779401849.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\ashdh\AppData\Local\Temp\ipykernel_14424\1779401849.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
#reading the data for the confirmed number of COVID cases and displaying them
confirmedCasesNew = pd.read_csv(r"..\..\DATASETS\COVID DATASETS\covid_confirmed_usafacts.csv")
a = confirmedCasesNew.iloc[:,4:].diff(axis=1).loc[:, '2022-05-30':'2023-01-01']
confirmedCases = pd.concat([confirmedCasesNew.iloc[:,:4], a], axis=1)
#confirmedCases


In [3]:
#reading the data for the confirmed number of COVID cases and displaying them
confirmedCasesNew = pd.read_csv(r"..\..\DATASETS\COVID DATASETS\covid_confirmed_usafacts.csv")
a = confirmedCasesNew.iloc[:,4:].diff(axis=1).loc[:, '2022-05-30':'2023-01-01']
confirmedCases = pd.concat([confirmedCasesNew.iloc[:,:4], a], axis=1)
confirmedCases



# reading the confirmed number of COVID Deaths and displaying them
confirmedDeathsNew = pd.read_csv(r"..\..\DATASETS\COVID DATASETS\covid_deaths_usafacts.csv")
b = confirmedDeathsNew.iloc[:,4:].diff(axis=1).loc[:, '2022-05-30':'2023-01-01']
confirmedDeaths = pd.concat([confirmedDeathsNew.iloc[:,:4], b], axis=1)


In [4]:
id_vars = ['countyFIPS', 'County Name', 'State', 'StateFIPS']
data1 = pd.melt(confirmedCases, id_vars= ['countyFIPS', 'County Name', 'State', 'StateFIPS'], var_name='date', value_name='cases')
data1 = data1[data1['County Name'] != 'Statewide Unallocated']

data2 = pd.melt(confirmedDeaths , id_vars= ['countyFIPS', 'County Name', 'State', 'StateFIPS'], var_name='date', value_name='deaths')
data2 = data2[data2['County Name'] != 'Statewide Unallocated']
data2

data = data1.merge(data2, on=['countyFIPS', 'County Name', 'State', 'StateFIPS', 'date'])


In [5]:
#creating log values
data['log_cases']=np.log(data['cases'])
data['log_deaths']=np.log(data['deaths'])

#making the date column as a date type
data['date']=data['date'].astype('datetime64[ns]')

#removing all negative and non numeric values from the cases and deaths
data['cases'] = data['cases'].clip(lower=0)
data['deaths'] = data['deaths'].clip(lower=0)
data['log_cases'] = data['log_cases'].clip(lower=0)
data['log_deaths'] = data['log_deaths'].clip(lower=0)

data

C:\Users\ashdh\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\ashdh\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\ashdh\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\ashdh\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,countyFIPS,County Name,State,StateFIPS,date,cases,deaths,log_cases,log_deaths
0,1001,Autauga County,AL,1,2022-05-30,9,0,2.197225,0.0
1,1003,Baldwin County,AL,1,2022-05-30,55,1,4.007333,0.0
2,1005,Barbour County,AL,1,2022-05-30,1,0,0.000000,0.0
3,1007,Bibb County,AL,1,2022-05-30,9,0,2.197225,0.0
4,1009,Blount County,AL,1,2022-05-30,6,0,1.791759,0.0
...,...,...,...,...,...,...,...,...,...
681809,56037,Sweetwater County,WY,56,2023-01-01,0,0,0.000000,0.0
681810,56039,Teton County,WY,56,2023-01-01,0,0,0.000000,0.0
681811,56041,Uinta County,WY,56,2023-01-01,0,0,0.000000,0.0
681812,56043,Washakie County,WY,56,2023-01-01,0,0,0.000000,0.0


In [6]:
data.info(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 681814 entries, 0 to 681813
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   countyFIPS   681814 non-null  int64         
 1   County Name  681814 non-null  object        
 2   State        681814 non-null  object        
 3   StateFIPS    681814 non-null  int64         
 4   date         681814 non-null  datetime64[ns]
 5   cases        681814 non-null  int64         
 6   deaths       681814 non-null  int64         
 7   log_cases    679581 non-null  float64       
 8   log_deaths   680011 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(2)
memory usage: 52.0+ MB


In [7]:
#ORIGINAL
app = dash.Dash(__name__)
df=data
states = data['State'].unique().tolist()
states.insert(0,"All States")
# Create dropdown options
dropdown_options = [{'label': state, 'value': state} for state in states]


app.layout = html.Div([
            html.H1("Covid-19 trend lines",style={'text-align':'center'}),
            dcc.DatePickerRange(
                id='my-date-picker-range',
                min_date_allowed=date(2022, 5, 30),
                max_date_allowed=date(2023, 1, 1),
                initial_visible_month=date(2022, 6, 1),
                start_date_placeholder_text='Start Date',
                end_date_placeholder_text='End Date',
                display_format='MM/DD/YYYY',
                month_format='MMMM, YYYY',
                start_date=date(2022, 6, 1),
                end_date=date(2023, 1, 1)
            ),
    html.Br(),
    
    #SETTING A DROP DOWN FOR STATE SELECTION
    html.Div([
        html.Label('Select State:'),
        dcc.Dropdown(
            id='state-dropdown',
            options=dropdown_options,
            #multi=True,
            value="All States"  # Set default value
            )]),
    html.Br(),
    #SETTING TOGGLE BUTTON FOR CASES/DEATHS
     html.Div([
            html.Label("Cases Or Deaths:"),
            dcc.RadioItems(id="CorD",
                options=[
                    {"label": "Cases", "value": "cases"},
                    {"label": "Deaths", "value": "deaths"}
                ],
                value="cases", labelStyle={"display":"block"})]),
    html.Br(),
       
    #SETTING TOGGLE FOR LINEAR/LOG  
    html.Div([
    html.Div([
            html.Label("Linear Or Log:"),
            dcc.RadioItems(id="LOrLog",
                options=[
                    {"label": "Linear", "value": "linear"},
                    {"label": "Log", "value": "log"}
                ],
                value="linear", labelStyle={"display":"block"})],
        style={'display': 'inline-block', 'margin-right': '20px'}),
    html.Br(),
    
    #SETTING CHECKLIST FOR LINEAR/NON-LINEAR REGRESSION LINE
    html.Div([
            html.Label("Regression: "),
            dcc.Checklist(id='regression-checklist',
                options=[            
                    {'label': 'Linear Regression', 'value': 'linearReg'},            
                    {'label': 'Nonlinear Regression', 'value': 'nonlinearReg'}
                ],
                value=[],
                inputClassName='toggle-radio',
                inputStyle={'margin-right': '10px'}
                         )
    ],
    style={'display': 'inline-block', 'margin-right': '20px'})
    ],
    style={'display': 'flex', 'flex-direction': 'row'}),
    dcc.Graph(id='Trend_Plot')
    ])



#NON REGRESSION PART
@app.callback(
    Output(component_id='Trend_Plot', component_property='figure'),
    [Input('my-date-picker-range', 'start_date'),
    Input('my-date-picker-range', 'end_date'),
    Input('CorD', 'value'),
    Input('LOrLog', 'value'),
    Input('state-dropdown', 'value'),
    Input('regression-checklist', 'value')]
)
def updated_graph(start_date, end_date, CorD_value, LorLog_value, selected_state,regression):
    df_state = df[df['State'] == selected_state]
    df_state = df_state.sort_values(by='date')
    df_state['log_cases'] = np.log(df_state['cases'])
    df_state['log_deaths'] = np.log(df_state['deaths'])
    if selected_state == "All States":
        df_state = data.groupby("date").sum().reset_index()
    else:
        df_state = data[data['State']== selected_state].groupby("date").sum().reset_index()
        '''dfs = {}
        for state in selected_state:
            dfs[state] = data[data['State'] == state].sort_values(by='date').groupby("date").sum().reset_index()
            #dfs[state] = data[data['State'].isin(selected_state)].groupby("date").sum().reset_index()
        df_state = pd.concat(dfs)
        '''

    if LorLog_value == 'linear':       
        if start_date is not None:
            df_1 = df_state[(df_state['date'] >= start_date) & (df_state['date'] <= end_date)]
        else:
            df_1 = df_state

        if CorD_value == 'cases':
            fig_c = px.scatter(df_1, x='date', y='cases', color='cases')
        elif CorD_value == 'deaths':
            fig_c = px.scatter(df_1, x='date', y='deaths', color='deaths')

    elif LorLog_value == 'log':
        if start_date is not None:
            df_1 = df_state[(df_state['date'] >= start_date) & (df_state['date'] <= end_date)]
        else:
            df_1 = df_state

        if CorD_value == 'cases':
            fig_c = px.scatter(df_1, x='date', y='log_cases', color='log_cases')
        elif CorD_value == 'deaths':
            fig_c = px.scatter(df_1, x='date', y='log_deaths', color='log_deaths')
    
    # Add regression lines if selected
    if "linearReg" in regression:
        X = np.array(df_1.index).reshape(-1, 1)
        y = np.array(df_1[CorD_value])
        model = LinearRegression()
        model.fit(X, y)
        y_pred = model.predict(X)
        fig_c.add_trace(go.Line(x=df_1['date'], y=y_pred, mode='lines', name='Linear Regression'))

    if "nonlinearReg" in regression:
        # Define a nonlinear regression function here
        X = np.array(df_1.index).reshape(-1, 1)
        y = np.array(df_1[CorD_value])
        poly = PolynomialFeatures(degree=4)
        X_poly = poly.fit_transform(X)
        model = LinearRegression()
        model.fit(X_poly, y)
        y_pred = model.predict(X_poly)
        fig_c.add_trace(go.Line(x=df_1['date'], y=y_pred, mode='lines', name='Polynomial Regression'))
        
        
    fig_c.update_traces(mode='lines+markers')
    fig_c.update_layout(
        title=f"{selected_state} COVID-19 {CorD_value.capitalize()} Trend ({start_date} to {end_date})",
        xaxis_title="Date",
        yaxis_title=f"{CorD_value.capitalize()}",
        legend_title=f"{CorD_value.capitalize()}",
    )

    return fig_c

# Run the app
if __name__ == '__main__':
    app.run_server(debug=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Apr/2023 19:10:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 19:10:37] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 19:10:37] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 19:10:38] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 19:10:38] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 19:10:38] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 19:10:38] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 19:10:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 19:10:41] "POST /_dash-update-component HTTP/1.1" 200 -
C:\Users\ashdh\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning:

divide by zero encountered in log

127.